### 参照設定

In [ ]:
#r "nuget:Microsoft.Data.Sqlite,8.0.3"
#r "nuget:Skender.Stock.Indicators,2.5.0"
#r "nuget:ccxt,4.2.85"
#r "nuget:ScottPlot,5.0.30"
#r "nuget:ReactiveProperty,9.5.0"
#r "/workspace/src/Common/obj/Debug/net8.0/Common.dll"
#r "/workspace/src/Domain/obj/Debug/net8.0/Domain.dll"
#r "/workspace/src/Infra/obj/Debug/net8.0/Infra.dll"

using Microsoft.Extensions.Logging;
using BotTrade.Infra;
using BotTrade.Infra.Exchanges;
using BotTrade.Domain;
using BotTrade.Domain.Strategies;


### 設定

In [ ]:
var setting = new Setting.Exchange() 
{ 
    Place = ExchangePlace.Binance, 
    Symbol = Symbol.Spot_BTCUSDT 
};


### DB更新

In [ ]:
var factory = LoggerFactory.Create(builder => 
{
    builder.AddConsole();
});
var logger = factory.CreateLogger<PastCandleRepository>();
var repository = new PastCandleRepository(setting, logger);

await repository.Fetch();


### バックアップ作成

In [ ]:
#!set --value @csharp:setting --name setting
$place = $null
switch($setting.RootElement.GetProperty("Place").GetInt64()) {
    0 { $place = "Bybit" }
#   1 { $place = "BybitTestnet" }
    2 { $place = "Binance" }
#   3 { $place = "BinanceTestnet" }
    default { echo "不明な取引所のためバックアップを行いません" }
}
if($place -eq $null) {
    return $null
}

$data_dir = "/workspace/data"
$db_file = "{0}/{1}.sqlite3" -f $data_dir, $place
$backup_file = "{0}/backup/{1}.backup" -f $data_dir, $place

sqlite3 $db_file .dump > $backup_file
